In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del_400_4_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1084503

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

8053240

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700394728
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 54225059


b'chr22\t3841\tT\tY\t+\nchr22\t3994\tC\tS\t+\nchr22\t6122\tTCC\t-\t-\nchr22\t6895\tC\tS\t+\nchr22\t7392\tC\t-\t-\nchr22\t8645\tC\tA\t-\nchr22\t8831\tG\tR\t+\nchr22\t8850\tA\tC\t-\nchr22\t9379\tC\tT\t-\nchr22\t11517\tC\tS\t+\nchr22\t11800\tT\tY\t+\nchr22\t11945\tT\tY\t+\nchr22\t12086\tA\tC\t-\nchr22\t13043\tT\tG\t-\nchr22\t14507\tA\t-\t-\nchr22\t15414\tC\tS\t+\nchr22\t15561\tA\tT\t-\nchr22\t15695\tG\tA\t-\nchr22\t15857\tC\tS\t+\nchr22\t16073\tG\t-\t-\nchr22\t17665\tT\tK\t+\nchr22\t18282\tG\tS\t+\nchr22\t19997\tG\tA\t-\nchr22\t20041\tG\tT\t-\nchr22\t22243\tA\tT\t-\nchr22\t22357\tA\tR\t+\nchr22\t22473\tC\tY\t+\nchr22\t24177\tG\tR\t+\nchr22\t24362\tC\tM\t+\nchr22\t25214\tG\tK\t+\nchr22\t26122\tG\tC\t-\nchr22\t26467\tA\tM\t+\nchr22\t29324\tT\tK\t+\nchr22\t30026\tA\tG\t-\nchr22\t31442\t-\tTGG\t+\nchr22\t33169\tC\tS\t+\nchr22\t33702\tC\tG\t-\nchr22\t34012\tA\tT\t-\nchr22\t34532\tC\t-\t-\nchr22\t37019\tT\tY\t+\nchr22\t37030\tT\tK\t+\nchr22\t37612\tT\tW\t+\nchr22\t40395\tT\tW\t+\nchr22\t40417\t

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 9 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4647	1.671180e-05	1.978923e-70	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.3779	0.000000e+00	6.653020e-275	0.000000e+00	4.778990e-245	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.1986	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4631	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854600	6300	1.6639	0.000000e+00	2.482440e-259	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	1.9473	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13449001-13453700	4700	1.3009	0.000000e+00	2.084793e-33	1.888859e-09	1.780490e-15	0.0000	1.0000	0
duplication	chr22:13480901-13512800	31900	4.3623	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:15242601-15246400	3800	3.0800	0.000000e+00	0.000000e+00	0.0000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4646573939034693,0.0
duplication,chr22,1306101,1325100,19000,1.3778587378657097,0.0
duplication,chr22,2926201,2963000,36800,3.198565326696644,0.0
duplication,chr22,6655001,6661800,6800,2.463077131484462,0.0
duplication,chr22,6848301,6854600,6300,1.663906406988916,0.0
duplication,chr22,8039401,8047900,8500,1.947253269064588,0.0
duplication,chr22,13449001,13453700,4700,1.300890660679861,0.0
duplication,chr22,13480901,13512800,31900,4.362267332885041,0.0
duplication,chr22,15242601,15246400,3800,3.079990370846542,0.0
duplication,chr22,18663101,18664600,1500,1.6172919338782117,0.0
duplication,chr22,18664801,18665900,1100,1.5742765009233157,0.0
duplication,chr22,18680801,18682700,1900,1.5199227647455225,0.0
duplication,chr22,18704901,18881400,176500,2.9852363377361555,0.0001607437533666411
duplication,chr22,19258601,19268300,9700,3.4361820381

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3787316076951632,0.0
duplication,chr22,2926001,2963000,37000,3.1908987425511426,0.0
duplication,chr22,6655001,6662000,7000,2.4288857948088314,0.0
duplication,chr22,8040001,8048000,8000,1.95278292153096,0.0
duplication,chr22,13481001,13513000,32000,4.353975783374236,0.0
duplication,chr22,18705001,18745000,40000,2.9397126406913845,0.00017592333881800214
duplication,chr22,18746001,18881000,135000,3.00569853909851,0.00013156572077319253
duplication,chr22,19259001,19268000,9000,3.497564626389971,0.0
duplication,chr22,20326001,20368000,42000,1.2983326336087957,0.000329574354720878
duplication,chr22,20609001,20655000,46000,1.4803182791370564,0.00023758507965699753
duplication,chr22,21466001,21505000,39000,1.336400260993155,0.00013776476666092647
duplication,chr22,21516001,21624000,108000,1.4504443105707612,0.0005089292205729854
duplication

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,18710001,18880000,170000,2.975768923203459,0.00014509114200883993
duplication,chr22,21460001,21680000,220000,1.345909876490098,0.0003692769637765531
duplication,chr22,24200001,24240000,40000,2.77948558655509,4.85525932479527e-05
duplication,chr22,25150001,25310000,160000,1.3406563595483147,0.00011537809261941632
duplication,chr22,28450001,28600000,150000,3.0569181819438445,4.376736188247402e-05
duplication,chr22,30800001,30980000,180000,3.2338356112136575,2.4641707013183828e-05
duplication,chr22,31820001,32250000,430000,1.2923392346239153,8.382562979377821e-05
duplication,chr22,35940001,36200000,260000,2.3842594938246546,3.6839518332992404e-05
duplication,chr22,36470001,36570000,100000,1.4348620366474223,0.00011260744627165086
duplication,chr22,43500001,43900000,400000,1.495996168657431,0.00026575766966222496
duplication,chr22,44150001,44470000,320000,2.

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_4_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app